# Le partitionnement des données

Le partitionnement (*clustering*) est une technique courante des statistiques multivariées pour effectuer des tâches de regroupement entre variables afin de révéler une structure sous-jacente. Il s’agit d’une méthode exploratoire qui aide à la classification des données en regroupant les individus dans des ensembles cohérents où la variance intra-groupes est minimisée quand la variance inter-groupes est, elle, maximisée.

Importons les modules qui seront nécessaires :

In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.cluster import hierarchy
from scipy.spatial.distance import pdist, squareform

## Une matrice de dissimilarité

### Avec des variables catégorielles

Prenons les réponses de cinq étudiant·es à un test comportant dix questions :

In [ ]:
n_students = 5
n_questions = 10

data = [
    [random.choice(['A', 'B', 'C', 'D']) for _ in range(n_questions)]
    for _ in range(n_students)
]

df = pd.DataFrame(data, index=[f"Student {i+1}" for i in range(n_students)], columns=[f'Q{i+1}' for i in range(0, n_questions)])

display(df)

Et faisons la comparaison deux à deux pour comptabiliser le nombre de fois où leurs réponses divergent. Enfin, normalisons en divisant le résultat par le nombre de questions afin d’obtenir un score entre 0 et 1 où 0 correspond à des étudiant·es aux réponses similaires et 1 des étudiant·es qui n’auront jamais répondu pareil :

In [ ]:
data_array = df.values

# calculate differences between rows
row_diffs = (data_array[:, None] != data_array).sum(axis=2)

# normalize
dissimilarity_matrix = row_diffs / n_questions

# matrix to a DataFrame
dissimilarity_df = pd.DataFrame(dissimilarity_matrix, index=df.index, columns=df.index)

display(dissimilarity_df)

Il est à présent facile d’identifier les paires d’étudiant·es dont les réponses se ressemblent le plus :

In [ ]:
# minimum score > 0 to exclude pairs consisting of the same student
min_score = dissimilarity_df[dissimilarity_df > 0].min().min()

# all the pairs concerned
clusters = dissimilarity_df[dissimilarity_df == min_score].stack().index.tolist()

display(clusters)

### Avec des variables numériques

Dans l’exemple précédent, les variables enregistraient des données catégorielles. Si maintenant nous prenons l’exemple d’une dizaine de textes avec des scores sur 20 dans cinq catégories :

In [ ]:
# 10 texts with a score on 5 categories
n_texts = 10
categories = ["Sciences", "Politique", "Littérature", "Journalisme", "Philosophie"]

df = pd.DataFrame(
    data=np.random.randint(0, 21, size=(n_texts, len(categories))),
    index=[f"Text {i + 1}" for i in range(0, n_texts)],
    columns=categories
)

display(df)

Il n’est plus question ici de repérer les catégories où les textes ont obtenu des scores différents, aussi la première étape consiste à calculer une matrice de corrélation :

In [ ]:
correlation_matrix = df.corr()

display(correlation_matrix)

Cette matrice ressort des coefficients variant de -1 à 1 pour exprimer la corrélation entre chaque paire de variables. Pour la transformer en une matrice de dissimilarité, il suffit de calculer l’inverse de la corrélation :

In [ ]:
dissimilarity_matrix = 1 / correlation_matrix

display(dissimilarity_matrix)

Une formule alternative consiste à calculer plutôt l’opposé de la corrélation :

In [ ]:
dissimilarity_matrix = 1 - correlation_matrix

display(dissimilarity_matrix)

Puis à normaliser afin d’obtenir des scores dans l’intervalle $[0,1]$ :

In [ ]:
display(dissimilarity_matrix / 2)

De là, nous pouvons effectuer des prédictions sur les *clusters* formés entre les catégories. Peut-être la littérature et la philosophie sont-elles liées par leur coefficient de dissimilarité et, comme la philosophie et la politique sont elles-mêmes reliées, pourrions-nous en conclure que les trois disciplines forment un groupe.

**Remarque :** les données sont générées aléatoirement aussi les groupes seront-ils toujours différents.

## Une matrice de distances

Bien souvent, lorsque l’on calcule la dissimilarité entre des vecteurs, on calcule la distance euclidienne :

$$
d(\mathbf{x}, \mathbf{y}) = \sqrt{\sum_{i=1}^{n} (x_i - y_i)^2}
$$

### Calcul de la distance euclidienne

La fonction peut s’interpréter en Python :

In [ ]:
def euclidean_distance(*, a:list, b:list) -> float:
    """Euclidean distance between two vectors.
    
    Keyword arguments:
    a -- first vector
    b -- second vector
    """
    # difference between indices
    coords = [
        (x - y) ** 2
        for x, y in zip(a, b)
    ]
    # distance = square root of the sum of coords
    return sum(coords) ** .5

Pour l’appliquer à notre jeu de données, on va d’abord générer une matrice de distances nulle avant de la remplir en se servant de la symétrie :

In [ ]:
# a null matrix
pairwise_distances = np.zeros((n_texts, n_texts))

for i in range(n_texts):
    for j in range(i, n_texts):  # start at 'i' to avoid calculating b-a pair if a-b already stored
        dist = euclidean_distance(a=df.iloc[i].values.tolist(), b=df.iloc[j].values.tolist())
        pairwise_distances[i, j] = dist
        pairwise_distances[j, i] = dist  # matrix is symetric

display(pairwise_distances)

### Calcul avec *Numpy*

Calculer la distance euclidienne entre deux vecteurs *a* et *b* revient à calculer la norme du vecteur différence $a - b$.

$$
\|\mathbf{v}\| = \sqrt{\sum_{i=1}^{n} v_i^2}
$$

Or, la fonction `.linalg.norm()` de *Numpy* permet d’obtenir directement la norme d’un vecteur ; aussi pouvons-nous nous passer de la fonction `euclidean_distance()` définie plus haut :

In [ ]:
# a null matrix
pairwise_distances = np.zeros((n_texts, n_texts))

for i in range(n_texts):
    # find vector a
    vector_a = df.iloc[i].values
    for j in range(i, n_texts):
        # find vector b
        vector_b = df.iloc[j].values
        dist = np.linalg.norm(vector_a - vector_b)
        pairwise_distances[i, j] = dist
        pairwise_distances[j, i] = dist

display(pairwise_distances)

### Une bibliothèque scientifique spécialisée

La bilbiothèque *Scipy* fournit des méthodes spécialisées pour le calcul scientifique. Utilisons les fonctions `pdist` et `squareform` du module `spatial.distance` :

In [ ]:
# euclidean matrix
distances = pdist(df.values, metric='euclidean')

# to a square matrix
distance_matrix = squareform(distances)

# to a dataframe
distance_df = pd.DataFrame(distance_matrix, index=df.index, columns=df.index)

display(distance_df)

## Le regroupement hiérarchique

Jusqu’ici, les techniques de partitionnement proposées étaient plutôt simples et élémentaires : elles permettaient de dégager des appariements, tout au plus des regroupements de trois éléments. L’objectif à présent est de présenter une méthode plus systématique qui va, à chaque étape, effectuer des regroupements deux à deux jusqu’à ce qu’il ne reste plus qu’un seul groupe constitué de l’ensemble des autres.

On peut matérialiser cette méthode grâce à un dendogramme :

In [ ]:
# by default, euclidean distance
Z = hierarchy.linkage(df, method='single')

# plot dendrogram
plt.figure(figsize=(15, 7))
plt.title('Dendrogram')
plt.xlabel('Samples')
plt.ylabel('Distance')
_ = hierarchy.dendrogram(
    Z, 
    labels=df.index, 
    leaf_rotation=90,
    leaf_font_size=10,
    color_threshold=0.7 * max(Z[:, 2])
)
plt.axhline(y=0.7 * max(Z[:, 2]), color='r', linestyle='--')
plt.show()

### Le clustering par liaison simple